# Notebook Testing Harness

This notebook executes all Jupyter notebooks in the project recursively and reports their completion status.

It serves as a testing harness to verify that all notebooks run successfully without errors.

In [ ]:
import os
import glob
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from datetime import datetime
import sys
import traceback
from pathlib import Path
import json
from tqdm.notebook import tqdm

from pyspark.conf import SparkConf #Recommended for making ALS run faster, if you have enough memory / cores allocated to docker
conf = SparkConf()
conf.set("spark.driver.memory", "7g")
conf.set("spark.executor.memory", "7g")
conf.set("spark.dynamicAllocation.enabled", "true")
conf.set("spark.dynamicAllocation.executorMemoryOverhead", "7g")

## Configuration

In [2]:
KERNEL_NAME = "python3"
KERNEL_OVERRIDES = {"1.open-information-extraction.ipynb": "ch5-spacy"}
EXCLUDE_DIRS = [".ipynb_checkpoints", "__pycache__"]
EXPORT_RESULTS = True
RESULTS_FILE = ""

## Utility Functions

In [3]:
def format_time(seconds):
    if seconds < 60:
        formatted = f"{seconds:.2f}s"
    elif seconds < 3600:
        formatted =  f"{seconds / 60:.2f}m"
    else:
        formatted =  f"{seconds / 3600:.2f}h"
    return formatted

def export_results(results, filename="test_results.json"):
    if results:
        with open(filename, 'w') as f:
            json.dump({"timestamp": datetime.now().isoformat(),
                       "results": results["details"],
                       "summary": results["summary"]}, f, indent=2)
        print(f"📄 Results exported to {filename}")

In [4]:
def get_notebook_files(root_dir=".", exclude_dirs=None, excluded_notebooks=None):
    """
    Recursively get all .ipynb files in the specified directory and its subdirectories,
    excluding specific directories and notebooks.
    
    Args:
        root_dir (str): Root directory to search from
        exclude_dirs (list): List of directory names to exclude
        exclude_notebooks (list): List of notebook filenames to exclude
        
    Returns:
        list: Sorted list of notebook file paths
    """
    if exclude_dirs is None:
        exclude_dirs = []
    
    if excluded_notebooks is None:
        excluded_notebooks = ["bonus.related-terms-from-documents.ipynb"
                              "bonus.phrase-detection.ipynb",
                              "bonus.phrase-detection.ipynb",
                              "bonus.related-terms-from-documents.ipynb",
                              "a.defunct.synthesize-search-sessions.ipynb",
                              "a.synthesize-search-sessions.ipynb",
                              "a.generate-movie-embeddings.ipynb",
                              "welcome.ipynb",
                              "aips-test-suite.ipynb",
                              "4.train-upload-search-ltr.ipynb"]
        
    notebook_files = []
    
    for directory in os.walk(root_dir):
        for path in Path(directory[0]).rglob("*.ipynb"):
            if any(exclude_dir in str(path) for exclude_dir in exclude_dirs) or \
                path.name in excluded_notebooks:
                continue
            if str(path) not in notebook_files:
                notebook_files.append(str(path))
    
    notebook_files = sorted(notebook_files)
    return list(map(Path, notebook_files))

In [5]:
def execute_notebook(notebook_path, timeout=600, kernel_name="python3"):
    start_time = datetime.now()
    
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
            ep = ExecutePreprocessor(timeout=timeout, kernel_name=kernel_name)
            ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_path)}})
        execution_time = (datetime.now() - start_time).total_seconds()
        return True, execution_time, None
    
    except Exception as e:
        execution_time = (datetime.now() - start_time).total_seconds()
        error_type = type(e).__name__
        error_msg = str(e)
        # Get traceback for detailed error information
        tb = traceback.format_exc()
        
        return False, execution_time, {
            'type': error_type,
            'message': error_msg,
            'traceback': tb
        }

## Main Testing Function

In [6]:
def run_test_harness(exclude_dirs=None, exclude_notebooks=None, stop_on_failure=False,
                     chapter_to_run=None, verbose_errors=True):
    """
    Run the testing harness on all notebooks recursively.
    
    Args:
        root_dir (str): Root directory to search from
        timeout (int): Execution timeout in seconds
        kernel_name (str): Name of the kernel to use
        exclude_dirs (list): List of directory names to exclude
        exclude_notebooks (list): List of notebook filenames to exclude
        
    Returns:
        dict: Test results
    """
    root_dir = "."
    timeout = 600 
    print(f"🔍 Notebook Testing Harness - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📁 Testing from root directory: {os.path.abspath(root_dir)}")
    print("=" * 80)

    notebook_files = get_notebook_files(root_dir, exclude_dirs, exclude_notebooks)

    if chapter_to_run:
        notebook_files = [f for f in notebook_files if (chapter_to_run in str(f))]

    print(f"📋 Found {len(notebook_files)} notebook(s) to test.")
    for nb_file in notebook_files:
        # Make paths relative to root_dir for cleaner display
        rel_path = os.path.relpath(str(nb_file), root_dir)
        print(f"   • {rel_path}")
    print()
    
    # Results tracking
    results = {
        'details': [],
        'summary': {
            'total': len(notebook_files),
            'successful': 0,
            'failed': 0,
            'total_time': 0
        }
    }
    
    # Execute each notebook with progress bar
    print("🚀 Executing notebooks:")
    for notebook_path in tqdm(notebook_files, desc="Progress", colour="purple"):
        rel_path = os.path.relpath(str(notebook_path), root_dir)
        print(f"\n📔 Testing: {rel_path}")
        if "checkpoint" in str(notebook_path):
            print(f"\n📔 Skipping: {rel_path}")
            continue
        kernel_name = KERNEL_OVERRIDES.get(notebook_path.name, "python3")
        success, execution_time, error = execute_notebook(str(notebook_path), timeout, kernel_name)
        
        results['summary']['total_time'] += execution_time
        
        if success:
            print(f"   ✅ SUCCESS - Completed in {format_time(execution_time)}")
            results['summary']['successful'] += 1
        else:
            print(f"   ❌ FAILED - Error after {format_time(execution_time)}")
            print(f"      Error: {error['type']}")
            results['summary']['failed'] += 1
        
        results['details'].append({
            'notebook': rel_path,
            'success': success,
            'execution_time': execution_time,
            'error': error
        })
        if not success and stop_on_failure:
            print("Terminating test run due to test failure.")
            break
    
    # Print summary
    print("\n" + "=" * 80)
    print(f"📊 SUMMARY:")
    print(f"   Total notebooks tested: {results['summary']['total']}")
    print(f"   ✅ Successful: {results['summary']['successful']}")
    print(f"   ❌ Failed: {results['summary']['failed']}")
    
    success_rate = (results['summary']['successful'] / results['summary']['total']) * 100
    print(f"   Success rate: {success_rate:.1f}%")
    print(f"   Total execution time: {format_time(results['summary']['total_time'])}")

    if results['summary']['failed'] > 0:
        print(f"\n❌ FAILED NOTEBOOKS:")
        for result in results['details']:
            if not result['success']:
                if verbose_errors:
                    print(f"   • {result['notebook']}: {result['error']['type']}: {result['error']['message']}")
                else:
                    print(f"   • {result['notebook']}: {result['error']['type']}")
    
    print(f"\n🏁 Testing completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if EXPORT_RESULTS and results:
        export_results(results, "AIPS_test_resulsts.json")
    
    return results

## Run the Testing Harness

In [7]:
results = run_test_harness(exclude_dirs=EXCLUDE_DIRS, stop_on_failure=True)

🔍 Notebook Testing Harness - 2025-07-28 17:11:35
📁 Testing from root directory: /home/jovyan
📋 Found 4 notebook(s) to test.
   • chapters/ch14/1.question-answering-visualizer.ipynb
   • chapters/ch14/2.question-answering-data-preparation.ipynb
   • chapters/ch14/3.question-answering-fine-tuning.ipynb
   • chapters/ch14/4.question-answering-demo-application.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/4 [00:00<?, ?it/s]


📔 Testing: chapters/ch14/1.question-answering-visualizer.ipynb
   ✅ SUCCESS - Completed in 20.37s

📔 Testing: chapters/ch14/2.question-answering-data-preparation.ipynb
   ✅ SUCCESS - Completed in 4.27m

📔 Testing: chapters/ch14/3.question-answering-fine-tuning.ipynb
   ❌ FAILED - Error after 4.29s
      Error: CellExecutionError
Terminating test run due to test failure.

📊 SUMMARY:
   Total notebooks tested: 4
   ✅ Successful: 2
   ❌ Failed: 1
   Success rate: 50.0%
   Total execution time: 4.68m

❌ FAILED NOTEBOOKS:
   • chapters/ch14/3.question-answering-fine-tuning.ipynb: CellExecutionError: An error occurred while executing the following cell:
------------------
#This method adopted from the following example notebook:
#https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
#Copyright 2021, Huggingface.  Apache 2.0 license.
import datasets

file = "data/question-answering/question-answering-training-set"
datadict = datasets.load_from_disk(file)

def

Progress:   0%|          | 0/2 [00:00<?, ?it/s]


📔 Testing: chapters/ch15/1.llm-exploration.ipynb
   ❌ FAILED - Error after 7.36s
      Error: CellExecutionError
Terminating test run due to test failure.

📊 SUMMARY:
   Total notebooks tested: 2
   ✅ Successful: 0
   ❌ Failed: 1
   Success rate: 0.0%
   Total execution time: 7.36s

❌ FAILED NOTEBOOKS:
   • chapters/ch15/1.llm-exploration.ipynb: CellExecutionError: An error occurred while executing the following cell:
------------------
r = get_generative_response("What is a unicorn?")
------------------

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[5], line 1
----> 1 r = get_generative_response("What is a unicorn?")

Cell In[2], line 9, in get_generative_response(prompt)
      4 response = mockedGenerativeResponses.loc[mockedGenerativeResponses["prompt"] == prompt, "response"].values
      5 if len(response) > 0:
      6     output = f'''
      7     <div style="overflow

## Export Results (Optional)